# Create NWBfile from Templeton dataset (single subject)
<br>Updated to run with pynwb 3.1

In [1]:
from datetime import datetime
from dateutil.tz import tzlocal
from glob import glob
import json
import os
from pathlib import Path
import sys
import time

import numpy as np
import pandas as pd
from PIL import Image
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.base import Images
from pynwb.behavior import BehavioralTimeSeries, PupilTracking
from pynwb.ecephys import ElectricalSeries, LFP
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.image import GrayscaleImage
from pynwb.misc import Units

from hdmf.backends.hdf5.h5_utils import H5DataIO

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

# Functions

In [4]:
def stim_log_Templeton_NWBstyle(EEGexp_stim_log, sweep_to_epoch_map):
    """
    This function takes an EEGexp-style stim_log and returns a table with more descriptive columns to create an NWB trial table.

    Inputs:
    -------
    EEGexp_stim_log : pandas.DataFrame
        The stim_log associated with a particular EEGexp object.
    sweep_to_epoch_map : list of str
        A list containing the behavioral epoch/state associated with each sweep defined in the stim_log.

    Outputs:
    --------
    trial_table: pandas.DataFrame
        A dataframe with specific columns to create the NWB trial table.
    column_desc_dict: dict
        A dictionary associating the column names to a description of the column information.
    
    """
    stim_type_map = {'biphasic': 'electrical', 'circle': 'visual', 'natural_scene': 'visual'}
    
    ## Initialize the NWBstyle trial table ##
    trial_table = pd.DataFrame({'start_time': EEGexp_stim_log['onset'].values, 'stop_time': EEGexp_stim_log['offset'].values})
    
    ## Adds extra columns describing stimuli ##
    stim_type = []
    stim_desc = []
    stim_curr = []
    natscene_ind = []
    for _, row in EEGexp_stim_log.iterrows():
        if row.stim_type == 'biphasic':
            stim_type.append(stim_type_map[row.stim_type])
            stim_desc.append(row.stim_type + '_pulse')
            stim_curr.append(int(row.parameter))
            natscene_ind.append(-1)
        elif row.stim_type == 'circle':
            stim_type.append(stim_type_map[row.stim_type])
            stim_desc.append(row.parameter + '_' + row.stim_type)
            stim_curr.append(-1)
            natscene_ind.append(-1)
        elif row.stim_type == 'natural_scene':
            stim_type.append(stim_type_map[row.stim_type])
            stim_desc.append(row.stim_type)
            stim_curr.append(-1)
            natscene_ind.append(int(row.parameter))
        else:
            stim_type.append('unknown')
            stim_desc.append('n/a')
            stim_curr.append(-1)
            natscene_ind.append(-1)
    trial_table['stimulus_type'] = stim_type
    trial_table['stimulus_description'] = stim_desc
    trial_table['estim_current'] = stim_curr
    trial_table['image_id'] = natscene_ind
    
    ## Add metadata for each trial ##
    trial_table['behavioral_epoch'] = [sweep_to_epoch_map[x] for x in EEGexp_stim_log['sweep'].values]
    
    ## Column descriptions ##
    column_desc_dict = {
        'stimulus_type': 'type of stimulus delivered',
        'stimulus_description': 'more specific description of stimulus type',
        'estim_current': 'electrical stimulation current (\u03bcA), if applicable',
        'image_id': 'index of natural scene image in NWBfile.acquisition.####, if applicable',
        'behavioral_epoch': 'behavioral epoch when stimulus was delivered',
    }
    
    return trial_table, column_desc_dict

# Start packaging

In [5]:
startclock = time.time()

Set paths

In [6]:
data_dir = Path(r"P:\\")
metadata_csv = Path(r"C:\Users\lesliec\OneDrive - Allen Institute\Shared Documents - Lab 328\Projects\Templeton\NWB packaging\all_Templeton_metadata.csv")
EEGinfo_csv = Path(r"C:\Users\lesliec\OneDrive - Allen Institute\Shared Documents - Lab 328\Projects\Templeton\NWB packaging\EEG_electrodes_info.csv")
NWB_savedir = Path(r"G:\TempletonNWB\testdata")
natural_scenes_dir = Path(r"C:\Users\lesliec\OneDrive - Allen Institute\Shared Documents - Lab 328\Projects\Templeton\NWB packaging\Natural Images")

Set metadata

In [7]:
institution = "Allen Institute" # MindScope Program?
# publications = "https://doi.org/10.7554/eLife.84630.1"
experimenters = [
    "Marks, Lydia C",
    "Claar, Leslie D",
    "Seyfourian, Parsa",
    "Rembado, Irene",
] # Can be a str or list
exp_desc = "in vivo electrophysiology with EEG and Neuropixels in a head-fixed mouse across psychedelic and anesthetized states"
keywords = ['EEG', 'Neuropixels', 'electrophysiology', 'psilocybin', 'anesthesia', 'urethane', 'isoflurane']

In [8]:
urethane_boost_buffer = 30. # s, time before/after urethane boost to exclude
saline_dose = '0.01 mL/g, i.p.'
psilocybin_dose = '1 mg/kg, i.p.'
ketanserin_dose = '1 mg/kg, i.p.'

## Load mouse/experiment metadata .csv file

In [9]:
subject_metadata = pd.read_csv(metadata_csv, dtype={'mouse': str}) # 'include': bool

In [10]:
subject_metadata.head()

,mouse,include,sex,DOB,strain,exp_folder,drug,sweep_state,NPX,EEG,estim,sensory,spontaneous,pupil_processed,bad_chs,first_inj,first_inj_window,second_inj,second_inj_window,urethane_dose_inj,urethane_boost,urethane_boost_inj,NPX_reference,EEG_reference
0,654182,True,F,9/13/2022,C57BL/6J,estim_vis_2022-12-01_10-33-50,saline,"awake,saline",True,True,"30,50,70",white circles,False,True,"3,6,14,22,23",2805.0,"2625, 2865",3423.0,"3243, 3483",NaN,NaN,NaN,Neuropixels tip reference electrode,"stainless steel skull screw, left CB"
1,654182,True,F,9/13/2022,C57BL/6J,urethane_vis_2022-12-02_11-02-25,urethane,urethane,True,True,"30,50,70",white circles,False,False,"3,6,14,22,23",NaN,NaN,NaN,NaN,"2.7 g/kg, i.v.",3180,"0.4 g/kg, i.p.",Neuropixels tip reference electrode,"stainless steel skull screw, left CB"
2,631037,True,M,4/23/2022,Grk4-Cre,estim_2022-12-06_09-54-04,saline,"awake,saline",True,True,"30,50,70",FALSE,False,True,"6,7,8,21",2197.0,"2017, 2257",2789.0,"2609, 2849",NaN,NaN,NaN,Neuropixels tip reference electrode,"stainless steel skull screw, left CB"
3,631037,True,M,4/23/2022,Grk4-Cre,urethane_2022-12-07_10-34-51,urethane,urethane,True,True,"30,50,70",FALSE,False,True,"6,7,8,21",NaN,NaN,NaN,NaN,"1.7 g/kg, i.p.",FALSE,NaN,Neuropixels tip reference electrode,"stainless steel skull screw, left CB"
4,655955,True,F,9/27/2022,C57BL/6J,estim_2022-12-13_10-05-19,saline,"awake,saline",True,True,"30,50,70",FALSE,False,True,NaN,2542.0,"2362, 2602",3146.0,"2966, 3206",NaN,NaN,NaN,Neuropixels tip reference electrode,"stainless steel skull screw, left CB"


## Load EEG electrodes locations

In [11]:
EEG_ch_info = pd.read_csv(EEGinfo_csv)

## Load single experiment

In [12]:
subind = 45
exprow = subject_metadata.iloc[subind]
print('{}: {}'.format(exprow.mouse, exprow.exp_folder))

692644: aw_psi_2023-09-07_11-51-57


In [13]:
if exprow.include:
    print('Loading EEGexp...')
    data_path = os.path.join(data_dir, 'mouse'+exprow.mouse, exprow.exp_folder, 'experiment1', 'recording1')
    exp = EEGexp(data_path, preprocess=False, make_stim_csv=False)
else:
    print('Not including subject/session.')

Loading EEGexp...
Experiment type: electrical stimulation


**Define session description, stimulus_notes, and pharmacology**

In [14]:
data_types = []
if exprow.EEG:
    data_types.append('EEG')
if exprow.NPX:
    data_types.append('Neuropixels')

stim_list = []
stim_desc_list = []
if exprow.spontaneous:
    stim_list.append('spontaneous')
    stim_desc_list.append('no external stimuli delivered')
else:
    if exprow.estim != 'FALSE':
        stim_list.append('electrical')
        stim_elec_file = os.path.join(exp.data_folder, r'stim_elec_location.json')
        if os.path.exists(stim_elec_file):
            with open(stim_elec_file) as elec_file:
                stim_elec_data = json.load(elec_file)
            stim_desc_list.append('single pulse electrical stimuli delivered to {}, CCF coordinates [{}]'.format(
                stim_elec_data['tip']['area'], ', '.join([str(x) for x in stim_elec_data['tip']['ccf_coords']]))
            )
        else:
            print('Stim electrode location file not found.')
            stim_desc_list.append('single pulse electrical stimuli targeted to deep layers of MOs')
    if exprow.sensory != 'FALSE':
        stim_list.append('visual')
        stim_desc_list.append('visual stimuli ({}) presented on monitor in front of right eye'.format(exprow.sensory))
    
session_desc = "{} experiment with {} ({})".format(exprow.drug, ' and '.join(data_types), ' and '.join(stim_list))
print('Session description:')
print(session_desc)
print('')
stim_notes = "; ".join(stim_desc_list)
print('Stimulus notes:')
print(stim_notes)

Session description:
psilocybin experiment with EEG and Neuropixels (electrical)

Stimulus notes:
single pulse electrical stimuli delivered to MOs6a, CCF coordinates [166, 109, 167]


In [15]:
if exprow.drug == 'psilocybin':
    pharma = "1st injection: saline ({}); 2nd injection: psilocybin ({})".format(saline_dose, psilocybin_dose)
elif exprow.drug == 'saline':
    pharma = "1st injection: saline ({}); 2nd injection: saline ({})".format(saline_dose, saline_dose)
elif exprow.drug == 'ketanserin+psilocybin':
    pharma = "1st injection: ketanserin ({}); 2nd injection: psilocybin ({})".format(ketanserin_dose, psilocybin_dose)
elif exprow.drug == 'isoflurane':
    pharma = "isoflurane (inhalation): induction at 5%, maintenance at 1.5%"
elif exprow.drug == 'urethane':
    pharma = "urethane (initial dose given before recording {})".format(exprow.urethane_dose_inj)
print(pharma)

1st injection: saline (0.01 mL/g, i.p.); 2nd injection: psilocybin (1 mg/kg, i.p.)


**Get session_start_time from sync file**

In [16]:
sync_data = exp._load_sync_dataset()
session_start_time = datetime.strptime(sync_data.meta_data['start_time'], '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=tzlocal())
print(session_start_time)

2023-09-07 11:50:54.189000-07:00


**Get subject age**

In [17]:
subject_DOB = datetime.strptime(exprow.DOB, '%m/%d/%Y').replace(tzinfo=tzlocal())
subject_age = "P{:d}D".format((session_start_time - subject_DOB).days)
print(subject_age)

P100D


**Create session id (mouse-expdate) and identifier**
<br>"identifier" should be unique to this NWBfile (and does not need to be easily human-readable)

In [18]:
session_id = "{}-{}".format(exp.mouse, session_start_time.strftime("%Y%m%d"))
file_id = session_id + "-test_pynwb28_v4noLFP"
print(file_id)

692644-20230907-test_pynwb28_v4noLFP


## Initialize the NWB file
Other optional fields exist, see: https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.

In [19]:
nwbfile = NWBFile(
    session_description = session_desc,
    identifier = file_id,
    session_start_time = session_start_time,
    experimenter = experimenters,
    experiment_description = exp_desc, # optional
    session_id = session_id,
    institution = institution,
    keywords = keywords,
    stimulus_notes = stim_notes,
    pharmacology = pharma,
)
print(nwbfile)

root pynwb.file.NWBFile at 0x1906078535872
Fields:
  experiment_description: in vivo electrophysiology with EEG and Neuropixels in a head-fixed mouse across psychedelic and anesthetized states
  experimenter: ['Marks, Lydia C' 'Claar, Leslie D' 'Seyfourian, Parsa' 'Rembado, Irene']
  file_create_date: [datetime.datetime(2025, 7, 24, 11, 10, 11, 637090, tzinfo=tzlocal())]
  identifier: 692644-20230907-test_pynwb28_v4noLFP
  institution: Allen Institute
  keywords: ['EEG' 'Neuropixels' 'electrophysiology' 'psilocybin' 'anesthesia'
 'urethane' 'isoflurane']
  pharmacology: 1st injection: saline (0.01 mL/g, i.p.); 2nd injection: psilocybin (1 mg/kg, i.p.)
  session_description: psilocybin experiment with EEG and Neuropixels (electrical)
  session_id: 692644-20230907
  session_start_time: 2023-09-07 11:50:54.189000-07:00
  stimulus_notes: single pulse electrical stimuli delivered to MOs6a, CCF coordinates [166, 109, 167]
  timestamps_reference_time: 2023-09-07 11:50:54.189000-07:00



## Add subject info
Other optional fields exist, see: https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject.

In [20]:
nwbfile.subject = Subject(
    age = subject_age,
    description = "mouse " + exp.mouse,
    sex = exprow.sex,
    species = "Mus musculus",
    subject_id = exp.mouse,
    strain = exprow.strain,
)

In [21]:
print(nwbfile)

root pynwb.file.NWBFile at 0x1906078535872
Fields:
  experiment_description: in vivo electrophysiology with EEG and Neuropixels in a head-fixed mouse across psychedelic and anesthetized states
  experimenter: ['Marks, Lydia C' 'Claar, Leslie D' 'Seyfourian, Parsa' 'Rembado, Irene']
  file_create_date: [datetime.datetime(2025, 7, 24, 11, 10, 11, 637090, tzinfo=tzlocal())]
  identifier: 692644-20230907-test_pynwb28_v4noLFP
  institution: Allen Institute
  keywords: ['EEG' 'Neuropixels' 'electrophysiology' 'psilocybin' 'anesthesia'
 'urethane' 'isoflurane']
  pharmacology: 1st injection: saline (0.01 mL/g, i.p.); 2nd injection: psilocybin (1 mg/kg, i.p.)
  session_description: psilocybin experiment with EEG and Neuropixels (electrical)
  session_id: 692644-20230907
  session_start_time: 2023-09-07 11:50:54.189000-07:00
  stimulus_notes: single pulse electrical stimuli delivered to MOs6a, CCF coordinates [166, 109, 167]
  subject: subject pynwb.file.Subject at 0x1906078532896
Fields:
  age

## Add running speed

In [22]:
## Set file names ##
running_file = os.path.join(exp.data_folder, 'running_signal.npy')
raw_running_file = os.path.join(exp.data_folder, 'raw_running_signal.npy')
running_ts_file = os.path.join(exp.data_folder, 'running_timestamps_master_clock.npy')

if os.path.exists(running_ts_file):
    print('Running signals already exist, loading.\n')
    raw_run_signal = np.load(raw_running_file)
    run_timestamps = np.load(running_ts_file)
else:
    print('Loading saved running signals.\n')
    run_signal, raw_run_signal, run_timestamps = exp.load_running()
    np.save(running_file, run_signal, allow_pickle=False)
    np.save(raw_running_file, raw_run_signal, allow_pickle=False)
    np.save(running_ts_file, run_timestamps, allow_pickle=False)

## Generate a TimeSeries for running speed ##
speed_with_samprate = TimeSeries(
    name = "running_speed",
    data = raw_run_signal,
    unit = "cm/s",
    starting_time = run_timestamps[0], # must be a float, not int
    rate = 100.0, # sampling rate must be a float
    description = "running speed data, computed from wheel angular velocity"
)

Running signals already exist, loading.



In [23]:
behavioral_time_series = BehavioralTimeSeries(
    time_series = speed_with_samprate, name = "BehavioralTimeSeries", # can rename, but should not
)
behavior_module = nwbfile.create_processing_module(name="behavior", description="processed behavioral data")
behavior_module.add(behavioral_time_series)

Data type,float64
Shape,"(589482,)"
Array size,4.50 MiB


## Add pupil radius

In [24]:
if exprow.pupil_processed:
    if len(glob(os.path.join(exp.data_folder, 'Pupileye*'), recursive=True)) > 0:
        print('Adding pupil metrics...')
        pupil_file = glob(os.path.join(exp.data_folder, 'Pupileye*'), recursive=True)[0]
        pupildf = pd.read_csv(pupil_file)
        ## Create TimeSeries for all pupil metrics ##
        pupil_center_x = TimeSeries(
            name = "pupil_center_x", data = pupildf.X_Position.values, unit = "pixels", timestamps = pupildf.sync_time.values,
            description = "x position of the center of an ellipse fit to the pupil, extracted from video of the right eye"
        )
        pupil_center_y = TimeSeries(
            name = "pupil_center_y", data = pupildf.Y_Position.values, unit = "pixels", timestamps = pupildf.sync_time.values,
            description = "y position of the center of an ellipse fit to the pupil, extracted from video of the right eye"
        )
        pupil_width = TimeSeries(
            name = "pupil_width", data = pupildf.Radius_1.values, unit = "pixels", timestamps = pupildf.sync_time.values,
            description = "semimajor (horizontal) axis of an ellipse fit to the pupil, extracted from video of the right eye"
        )
        pupil_height = TimeSeries(
            name = "pupil_height", data = pupildf.Radius_2.values, unit = "pixels", timestamps = pupildf.sync_time.values,
            description = "semiminor (vertical) axis of an ellipse fit to the pupil, extracted from video of the right eye"
        )
        pupil_phi = TimeSeries(
            name = "pupil_phi", data = pupildf.Angle.values, unit = "radians", timestamps = pupildf.sync_time.values,
            description = "tilt (angle the semimajor axis makes with the x-axis) of an ellipse fitted to the pupil, extracted from video of the right eye"
        )
        eye_width = TimeSeries(
            name = "eye_width", data = pupildf.Eye_Diameter.values, unit = "pixels", timestamps = pupildf.sync_time.values,
            description = "width of the eye, extracted from video of the right eye"
        )
        ## Add all TimeSeries to NWBfile ##
        pupil_tracking = PupilTracking(time_series=[pupil_center_x, pupil_center_y, pupil_width, pupil_height, pupil_phi, eye_width], name="PupilTracking")
        behavior_module.add(pupil_tracking)
    
    else:
        print('No local Pupileye*.csv file. Check server.')
        pupil_files = glob(os.path.join(exp.data_folder, 'Pupil*'), recursive=True)
        print(pupil_files)
else:
    print('False in the pupil_processed column.')

Adding pupil metrics...


### What do we do if there is no Pupileye*.csv file? All good videos should have eye processing, check them.

#### Testing pupil metrics

Generate TimeSeries for pupil_center_x, pupil_center_y, pupil_width, pupil_height, pupil_phi, eye_width

## Add stimulus log info into trial table

In [25]:
if exprow.spontaneous:
    print('Spontaneous recording, no stim log.')
else:
    print(stim_list)
    stim_log = pd.read_csv(exp.stimulus_log_file).astype({'parameter': str})

    ## Get state label for each sweep ##
    statelist = []
    for char in exprow.sweep_state.split(','):
        statelist.append(char)
    if len(np.unique(stim_log['sweep'].values)) > len(statelist):
        statelist = statelist * len(np.unique(stim_log['sweep'].values))

    ## Take stim_log and add NWBstyle info to it ##
    trial_table, col_desc_map = stim_log_Templeton_NWBstyle(stim_log, statelist)

    ## Add new columns to trial table ##
    for coln, cold in col_desc_map.items():
        nwbfile.add_trial_column(name = coln, description = cold)
        
    ## Now add each trial ##
    for _, row in trial_table.iterrows():
        row_dict = row.to_dict()
        nwbfile.add_trial(**row_dict)

    ## Add the natural scene images to the file. These are already grayscale images. ##
    if 'natural_scene' in np.unique(trial_table['stimulus_description'].values):
        print('There are natural scenes in this session, adding them to NWBfile.')
        nats_trials = trial_table[trial_table['stimulus_description'] == 'natural_scene']
        natural_scenes_files = os.listdir(natural_scenes_dir)
        gsimage_list = []
        for imageind in np.unique(nats_trials.image_id.values):
            img = Image.open(os.path.join(natural_scenes_dir, natural_scenes_files[imageind]))
            gsimage_list.append(
                GrayscaleImage(name=str(imageind), data=np.array(img), description="filename: " + natural_scenes_files[imageind]))
        natural_scene_images = Images(
            name="natural_scenes", images=gsimage_list, description="the collection of natural scenes presented to the subject")
        # nwbfile.add_acquisition(natural_scene_images)
        nwbfile.add_stimulus(natural_scene_images)
    else:
        print('There are NOT natural scenes in this session.')

['electrical']
There are NOT natural scenes in this session.


#### Testing stim_log

**Take stim_log and add NWBstyle info to it**

**Add trials to NWBfile**

## Drug information
<br>For saline, psilocybin, and ketanserin, include both injection **windows**.
<br>For isoflurane, IR wants to include the isoflurane signal, rather than epochs.
<br>For urethane, include the boost injection window, if applicable.

In [26]:
if exprow.drug in ['psilocybin', 'saline', 'ketanserin+psilocybin']:
    print('Getting injection times...')
    first_inj = []
    for char in exprow.first_inj_window.split(', '):
        first_inj.append(float(char))
    second_inj = []
    for char in exprow.second_inj_window.split(', '):
        second_inj.append(float(char))
    if exprow.drug == 'psilocybin':
        print(' Adding injection epochs for psilocybin.')
        nwbfile.add_epoch(start_time = first_inj[0], stop_time = first_inj[1], tags = ['1st injection', 'saline', saline_dose])
        nwbfile.add_epoch(start_time = second_inj[0], stop_time = second_inj[1], tags = ['2nd injection', 'psilocybin', psilocybin_dose])
    elif exprow.drug == 'saline':
        print(' Adding injection epochs for saline.')
        nwbfile.add_epoch(start_time = first_inj[0], stop_time = first_inj[1], tags = ['1st injection', 'saline', saline_dose])
        nwbfile.add_epoch(start_time = second_inj[0], stop_time = second_inj[1], tags = ['2nd injection', 'saline', saline_dose])
    elif exprow.drug == 'ketanserin+psilocybin':
        print(' Adding injection epochs for ketanserin+psilocybin.')
        nwbfile.add_epoch(start_time = first_inj[0], stop_time = first_inj[1], tags = ['1st injection', 'ketanserin', ketanserin_dose])
        nwbfile.add_epoch(start_time = second_inj[0], stop_time = second_inj[1], tags = ['2nd injection', 'psilocybin', psilocybin_dose])
elif exprow.drug == 'isoflurane':
    print('Getting isoflurane signal...')
    iso_level, isots = exp.load_analog_iso()
    iso_level = iso_level - np.mean(iso_level[:300])
    ## Generate a TimeSeries for isoflurane signal ##
    isoflurane_level = TimeSeries(
        name = "isoflurane_level",
        data = iso_level,
        unit = "%",
        starting_time = isots[0], # must be a float, not int
        rate = 100.0, # sampling rate must be a float
        description = "isoflurane level (%), applied via nose cone"
    )
    ## Add isoflurane signal to acquisition ##
    nwbfile.add_acquisition(isoflurane_level)
elif exprow.drug == 'urethane':
    if exprow.urethane_boost == 'FALSE':
        print('There was no boost injection.')
    else:
        print('Yes, there was a boost at {:.1f} s, adding injection epoch.'.format(float(exprow.urethane_boost)))
        nwbfile.add_epoch(
            start_time = float(exprow.urethane_boost) - urethane_boost_buffer,
            stop_time = float(exprow.urethane_boost) + urethane_boost_buffer,
            tags = ['urethane boost injection', exprow.urethane_boost_inj]
        )

Getting injection times...
 Adding injection epochs for psilocybin.


# Neural data

Add extra columns to electrodes table

In [28]:
nwbfile.add_electrode_column(name = "is_data_valid", description = "True for electrodes/channels with usable data")
# Specifically for NPXs, must add here though #
nwbfile.add_electrode_column(name = "probe_vertical_position", description = "length-wise position of electrode/channel on device (\u03bcm)")
nwbfile.add_electrode_column(name = "probe_horizontal_position", description = "width-wise position of electrode/channel on device (\u03bcm)")

## Add EEG

In [29]:
if exprow.EEG:
    print('Adding EEG to NWBfile...')
    ## Add EEG device ##
    eegdevice = nwbfile.create_device(name = "EEG array", description = "H32 Mouse EEG (30-ch)", manufacturer = "Neuronexus")
    ## Add 30 electrodes in one group ##
    EEG_elec_group = nwbfile.create_electrode_group(
        name = "EEG array", description = "30-ch surface grid", device = eegdevice, location = "skull surface, both hemispheres",
    )
    ## Get EEG bad chs as list ##
    EEG_bad_chs = []
    if type(exprow.bad_chs) == str:
        for char in exprow.bad_chs.split(','):
            EEG_bad_chs.append(int(char))
    print(EEG_bad_chs)
    ch_valid = [True if x not in EEG_bad_chs else False for x in range(len(EEG_ch_info))]
    ## Add EEG electrodes to NWBfile ##
    for chi, row in EEG_ch_info.iterrows():
        nwbfile.add_electrode(
            x = float(row.x),
            y = float(row.y),
            z = float(row.z),
            location = row.location,
            group = EEG_elec_group,
            reference = exprow.EEG_reference,
            is_data_valid = ch_valid[chi],
            probe_vertical_position = -1, # "n/a" could be causing a problem with other float values
            probe_horizontal_position = -1,
        )
    ## Create a DynamicTableRegion for the EEG electrodes ##
    EEGchs = list(np.arange(len(EEG_ch_info)))
    EEG_table_region = nwbfile.create_electrode_table_region(region = EEGchs, description = "EEG electrodes")
    ## EEG data and timestamps ##
    eeg_data = exp._memmap_EEGdata()
    eeg_times = np.load(exp.ephys_params['EEG']['timestamps'])
    ## With H5DataIO compression ##
    EEG_electrical_series = ElectricalSeries(
        name = "ElectricalSeriesEEG", # this is the key used to call up the data within the NWBfile, BP: ElectricalSeriesFromEEG, etc.
        data = H5DataIO(data=eeg_data, compression=True),
        electrodes = EEG_table_region,
        timestamps = H5DataIO(data=eeg_times, compression=True),
    #     resolution = , # float, smallest meaningful difference between values in data (in V)
        conversion = exp.ephys_params['EEG']['bit_volts'] * 1E-6, # float, scalar to multiply data by to convert it to Volts
        description = "voltage measured over time and associated timestamps from EEG array",
    )
    nwbfile.add_acquisition(EEG_electrical_series)
else:
    print('False in the EEG column.')

Adding EEG to NWBfile...
[29]


#### Testing adding EEG data

**Add EEG device**

**Add 30 electrodes in one group**

*All electrodes will be added to the same DynamicTable, EEG and all NPXs.* Will need to keep track, so the DynamicTableRegion can be assigned accordingly.

**Create a DynamicTableRegion for the EEG electrodes**

**Add EEG data with timestamps**

Test the file

## Add NPX to electrodes table

In [30]:
if 'Neuropixels' in data_types:
    print('This recording has NPX.')
    probe_list = [x.replace('_sorted', '') for x in exp.experiment_data if 'probe' in x]
    print(probe_list)
else:
    print('Skip NPX section.')

This recording has NPX.
['probeB', 'probeC', 'probeD', 'probeF']


In [31]:
probe_locs = np.ones((len(probe_list)), dtype=bool)
for pbi, probei in enumerate(probe_list):
    with open(exp.ephys_params[probei]['probe_info']) as data_file:
        data = json.load(data_file)
    if 'area_ch' not in data.keys():
        probe_locs[pbi] = False
if ~probe_locs.any():
    print(' This experiment has no probe locations, not adding NPX devices to NWBfile.\n')

In [32]:
unit_col_desc_map = {
    'location': 'the location of unit within the brain (CCF acronym)',
    'x': 'the x coordinate of the position (CCF, +x is posterior)',
    'y': 'the y coordinate of the position (CCF, +y is inferior)',
    'z': 'the z coordinate of the position (CCF, +z is right)',
    'isi_violations': 'an estimate of the relative firing rate of hypothetical neurons generating inter-spike-interval violations',
    'amplitude_cutoff': 'an estimate of the fraction of spikes below the spike detection threshold',
    'presence_ratio': 'the fraction of time the unit was present during the experiment (ranges from 0 to 0.99)',
    'waveform_duration': 'the mean duration (s) of detected spiking events',
}

for coln, cold in unit_col_desc_map.items():
    nwbfile.add_unit_column(name = coln, description = cold)

In [33]:
## Take out LFP for memory issues ##
try:
    electrode_counter = len(nwbfile.electrodes)
except TypeError:
    electrode_counter = 0

probe_NWB_objects = {}

for pbi, probei in enumerate(probe_list):
    if probe_locs[pbi]:
        print(probei)
        probe_NWB_objects[probei] = {}

        ## Create device ##
        probe_NWB_objects[probei]['device'] = nwbfile.create_device(
            name = probei, description = "Neuropixels 1.0 probe", manufacturer = "imec",
        )

        ## Create electrode group ##
        probe_NWB_objects[probei]['elec_group'] = nwbfile.create_electrode_group(
            name = probei,
            description = "Neuropixels probe " + probei[-1],
            device = probe_NWB_objects[probei]['device'],
            location = "see electrode locations",
        )

        ## Load probe_info.json ##
        with open(exp.ephys_params[probei]['probe_info']) as data_file:
            data = json.load(data_file)
        npx_allch = np.array(data['channel'])
        surface_ch = int(data['surface_channel'])
        ref_mask = np.array(data['mask'])
        npxch_valid = [True if ref_mask[x] and x <= surface_ch else False for x in npx_allch]
        vert_pos = np.array(data['vertical_pos'])
        horiz_pos = np.array(data['horizontal_pos'])
        ch_locs = np.array(data['area_ch'])
        ccf_coords = np.array(data['ccf_coord_ch']).astype(float)

        for npxchi in npx_allch:
            nwbfile.add_electrode(
                x = ccf_coords[npxchi, 0],
                y = ccf_coords[npxchi, 1],
                z = ccf_coords[npxchi, 2],
                location = ch_locs[npxchi],
                group = probe_NWB_objects[probei]['elec_group'],
                reference = exprow.NPX_reference,
                is_data_valid = npxch_valid[npxchi],
                probe_vertical_position = vert_pos[npxchi],
                probe_horizontal_position = horiz_pos[npxchi],
            )
            
        ## Create probe DynamicTableRegion ##
        probechs = list(np.arange(electrode_counter, electrode_counter + len(npx_allch)))
        probe_NWB_objects[probei]['elec_table_region'] = nwbfile.create_electrode_table_region(
            region = probechs, description = "{} electrodes".format(probei),
        )
        electrode_counter += len(npx_allch)
        
        ## Add the LFP data ##
        # lfp_ts = np.load(exp.ephys_params[probei]['lfp_timestamps'])
        # lfp_mm = np.memmap(
        #     exp.ephys_params[probei]['lfp_continuous'], dtype='int16',
        #     shape=(lfp_ts.size, exp.ephys_params[probei]['num_chs']), mode='r'
        # )
        
        ## Method with H5DataIO compression ##
        # probe_NWB_objects[probei]['ElecSeries'] = ElectricalSeries(
        #     name = "ElectricalSeries" + probei, # BP: ElectricalSeriesFromEEG, etc.
        #     data = H5DataIO(data=lfp_mm, compression=True),
        #     electrodes = probe_NWB_objects[probei]['elec_table_region'],
        #     timestamps = H5DataIO(data=lfp_ts, compression=True),
        #     conversion = exp.ephys_params[probei]['bit_volts'] * 1E-6,
        #     description = "voltage measured over time and associated timestamps from Neuropixels" + probei,
        # )
        # probe_NWB_objects[probei]['LFPSeries'] = LFP(
        #     electrical_series = probe_NWB_objects[probei]['ElecSeries'], name = "LFP" + probei
        # )
        # nwbfile.add_acquisition(probe_NWB_objects[probei]['LFPSeries'])

        ## Add probe units ##
        cluster_group = pd.read_csv(exp.ephys_params[probei]['cluster_group'], sep='\t')
        cluster_metrics = pd.read_csv(exp.ephys_params[probei]['cluster_metrics'])
        all_spike_times = np.load(exp.ephys_params[probei]['spike_times'])
        all_spike_clusters = np.load(exp.ephys_params[probei]['spike_clusters'])
        all_spike_waveforms = np.load(exp.ephys_params[probei]['waveforms']) # [clusters, chs, time] (uV)

        if not np.array_equal(cluster_group['cluster_id'].values.astype('int'), np.unique(all_spike_clusters)):
            print(' IDs from cluster_group.tsv DO NOT match spike_clusters.npy. This may mean there are unsorted units, check in phy.')
            continue
        if np.array_equal(cluster_group['cluster_id'].values.astype('int'), cluster_metrics['cluster_id'].values.astype('int')):
            all_unit_metrics = pd.merge(cluster_group.rename(columns={'group':'label'}), cluster_metrics, on='cluster_id')
        else:
            print(' IDs from cluster_group DO NOT match cluster_metrics.')
            continue

        ## Choose all "good" labeled units ##
        unit_metrics = all_unit_metrics[all_unit_metrics['label'] == 'good']
        for ii, (indi, row) in enumerate(unit_metrics.iterrows()):
            uniti = row.cluster_id
            spikesi = all_spike_times[all_spike_clusters == uniti]
            if len(spikesi) < 2:
                continue
            if isinstance(row.area, float):
                continue
            unit_coords = [int(x) for x in row.ccf_coord.replace('[','').replace(']','').replace(' ','').split(',')]

            ## Add unit to NWBfile ##
            nwbfile.add_unit(
                spike_times = spikesi,
                electrodes = [probe_NWB_objects[probei]['elec_table_region'][row.peak_channel].index[0]],
                electrode_group = probe_NWB_objects[probei]['elec_group'],
                waveform_mean = all_spike_waveforms[uniti, row.peak_channel, :] * 1E-6, # now in V
                location = row.area,
                x = unit_coords[0],
                y = unit_coords[1],
                z = unit_coords[2],
                isi_violations = row.isi_viol,
                amplitude_cutoff = row.amplitude_cutoff,
                presence_ratio = row.presence_ratio,
                waveform_duration = row.duration,
            )

    else:
        print(' This probe has no locations, not adding it to NWBfile.\n')


probeB
probeC
probeD
probeF


## Check the file

In [34]:
midclock = time.time()
print('Time to package: {:.2f} s'.format(midclock - startclock))

Time to package: 133.62 s


In [35]:
nwbfile

Data type,float64
Shape,"(639695,)"
Array size,4.88 MiB
Data type,float64
Shape,"(191518,)"
Array size,1.46 MiB
Data type,float64
Shape,"(191518,)"
Array size,1.46 MiB
Data type,float64
Shape,"(191518,)"


Look at the electrodes for non-string elements

Look at the units table

## Write the NWBfile

Once you have finished adding all of your data to the :py:class:`~pynwb.file.NWBFile`,
write the file with :py:class:`~pynwb.NWBHDF5IO`.

In [36]:
my_test_file = os.path.join(NWB_savedir, file_id + r'.nwb')
print(my_test_file)

G:\TempletonNWB\testdata\676727-20230511-test_pynwb28_v4noLFP.nwb


In [37]:
with NWBHDF5IO(my_test_file, "w") as io:
    io.write(nwbfile, cache_spec=True)
stopclock = time.time()
print('Time to write NWB file: {:.2f} min'.format((stopclock - midclock) / 60))

Time to write NWB file: 2.74 min
